In [ ]:
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import os

# --- Load models ---
yolo_model = YOLO('best.pt')  # Use your trained YOLOv8 model
cnn_model = load_model('astrocyte_live_dead_model.h5')   # Your trained CNN model

# --- Parameters ---
input_image_path = 'test5.tif'
confidence_threshold = 0.3
cnn_input_size = (128, 128)  # Resize input for CNN
class_labels = ['Dead', 'Live']

# --- Load image ---
image = cv2.imread(input_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
original = image.copy()

# --- Run YOLOv8 prediction ---
results = yolo_model.predict(source=image, conf=confidence_threshold, save=False)[0]
boxes = results.boxes.data.cpu().numpy() if results.boxes is not None else []

for box in boxes:
    x1, y1, x2, y2, conf, cls = box
    if conf < confidence_threshold:
        continue  # skip boxes below confidence threshold

    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
    cropped_cell = image[y1:y2, x1:x2]

    # --- Convert to RGB ---
    if cropped_cell.size == 0:
        continue  # skip if box is out of bounds or empty
    cropped_cell = cv2.cvtColor(cropped_cell, cv2.COLOR_BGR2RGB)

    # --- Resize and normalize ---
    cell_resized = cv2.resize(cropped_cell, cnn_input_size)
    cell_array = img_to_array(cell_resized).astype("float32") / 255.0
    cell_array = np.expand_dims(cell_array, axis=0)

    # --- CNN prediction ---
    threshold = 0.3  # You can lower this to be more sensitive to 'Live'
    prediction = cnn_model.predict(cell_array, verbose=0)
    prob = prediction[0][0]
    label = 'Live' if prob >= threshold else 'Dead'


    # --- Choose color ---
    color = (0, 255, 0) if label == 'Live' else (0, 0, 255)

    # --- Draw result ---
    cv2.rectangle(original, (x1, y1), (x2, y2), color, 2)
    cv2.putText(original, f'{label} ({conf:.2f})', (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)



# --- Display and save ---
cv2.imwrite('output_results5.jpg', original)
cv2.imshow('Live/Dead Cell Detection', original)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 416x640 29 cells, 186.3ms
Speed: 10.2ms preprocess, 186.3ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 640)


In [5]:
cnn_model.summary()



Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 126, 126, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 61, 61, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_18 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 28, 28, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,667 (12.61 MB)

 Trainable params: 3,305,217 (12.61 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 2 (12.00 B)

In [15]:
print(f"Box @ ({x1},{y1},{x2},{y2}) -> {label} | Conf: {conf:.2f}")


Box @ (974,353,1122,494) -> Dead | Conf: 0.59
